Доброго времени суток. На этой неделе защищаю диплом, поэтому не получилось уделить достаточно времени для изучения темы. Сделала ЛР по имеющимся знаниям и опыту. Не уверена в правильности.

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention
import tensorflow as tf

Загрузка данных

№Была использована нейронная сеть Seq2Seq с механизмом Attention для обучения переводчика с русского на английский язык, используя доступные данные.

In [ ]:
file_path = 'Rus-Eng-small.tsv'

# Чтение файла в DataFrame
df = pd.read_csv(file_path, delimiter='\t')

# Отображение первых нескольких строк DataFrame
print(df.head())


    243 Один раз в жизни я делаю хорошее дело... И оно бесполезно.     3257  \
0  5409                      Давайте что-нибудь попробуем!             1276   
1  5410                               Мне пора идти спать.             1277   
2  5411                                    Что ты делаешь?            16492   
3  5411                                    Что ты делаешь?           511884   
4  5411                                    Что ты делаешь?          1839618   

  For once in my life I'm doing a good deed... And it is useless.  
0                               Let's try something.               
1                             I have to go to sleep.               
2                                What are you doing?               
3                                  What do you make?               
4                                 What're you doing?               


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Предобработка данных
def preprocess_sentences(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.lower().strip()
        sentence = '<start> ' + sentence + ' <end>'
        cleaned_sentences.append(sentence)
    return cleaned_sentences

# Извлечение данных
sample_size = 25000
df_sample = df.sample(n=sample_size, random_state=42)

rus_sentences = df_sample.iloc[:, 1].astype(str).values
eng_sentences = df_sample.iloc[:, 2].astype(str).values

# Очистка и добавление токенов
rus_sentences = preprocess_sentences(rus_sentences)
eng_sentences = preprocess_sentences(eng_sentences)

# Токенизация
rus_tokenizer = Tokenizer()
eng_tokenizer = Tokenizer()

rus_tokenizer.fit_on_texts(rus_sentences)
eng_tokenizer.fit_on_texts(eng_sentences)

rus_sequences = rus_tokenizer.texts_to_sequences(rus_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(eng_sentences)

max_rus_len = max(len(seq) for seq in rus_sequences)
max_eng_len = max(len(seq) for seq in eng_sequences)

rus_sequences = pad_sequences(rus_sequences, maxlen=max_rus_len, padding='post')
eng_sequences = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')

rus_vocab_size = len(rus_tokenizer.word_index) + 1
eng_vocab_size = len(eng_tokenizer.word_index) + 1

print(f"Форма русских последовательностей: {rus_sequences.shape}")
print(f"Форма английских последовательностей: {eng_sequences.shape}")


Форма русских последовательностей: (25000, 115)
Форма английских последовательностей: (25000, 3)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Attention
import tensorflow as tf

embedding_dim = 256
units = 512

# Энкодер
encoder_inputs = Input(shape=(max_rus_len,))
encoder_embedding = Embedding(input_dim=rus_vocab_size, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

# Декодер
decoder_inputs = Input(shape=(max_eng_len - 1,))
decoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

# Механизм внимания
attention = Attention()
context_vector = attention([decoder_outputs, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_outputs])

# Полносвязный слой
decoder_dense = Dense(eng_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

# Модель
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

decoder_input_data = eng_sequences[:, :-1]
decoder_target_data = eng_sequences[:, 1:]

# Обучение модели
batch_size = 64
epochs = 5

history = model.fit(
    [rus_sequences, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)


Epoch 1/5
313/313 [==============================] - 1214s 4s/step - loss: 5.4812 - accuracy: 0.4887 - val_loss: 5.2816 - val_accuracy: 0.5000
Epoch 2/5
313/313 [==============================] - 1193s 4s/step - loss: 5.0967 - accuracy: 0.4999 - val_loss: 5.6676 - val_accuracy: 0.5000
Epoch 3/5
313/313 [==============================] - 1191s 4s/step - loss: 4.9875 - accuracy: 0.5001 - val_loss: 5.9047 - val_accuracy: 0.5000
Epoch 4/5
313/313 [==============================] - 1176s 4s/step - loss: 4.8788 - accuracy: 0.5000 - val_loss: 6.2152 - val_accuracy: 0.5000
Epoch 5/5
313/313 [==============================] - 1180s 4s/step - loss: 4.6399 - accuracy: 0.5003 - val_loss: 6.4104 - val_accuracy: 0.5000


Декодирование последовательностей и тестирование модел

In [ ]:
# Создание модели энкодера для декодирования
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

# Создание модели декодера для декодирования
decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_hidden_state_input = Input(shape=(max_rus_len, units))

decoder_embedding2 = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim)
decoder_inputs2 = Input(shape=(None,))
decoder_embedding_outputs = decoder_embedding2(decoder_inputs2)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    decoder_embedding_outputs, initial_state=[decoder_state_input_h, decoder_state_input_c])

context_vector2 = attention([decoder_outputs2, decoder_hidden_state_input])
decoder_combined_context2 = Concatenate(axis=-1)([context_vector2, decoder_outputs2])
decoder_outputs2 = decoder_dense(decoder_combined_context2)

decoder_model = Model(
    [decoder_inputs2, decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2, state_h2, state_c2])


In [ ]:
  def decode_sequence(input_seq):
      encoder_outputs, h, c = encoder_model.predict(input_seq)

      target_seq = np.zeros((1, 1))
      target_seq[0, 0] = eng_tokenizer.word_index.get('<start>', 0)

      stop_condition = False
      decoded_sentence = ''
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict(
              [target_seq, encoder_outputs, h, c])

          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_word = eng_tokenizer.index_word.get(sampled_token_index, '')

          if sampled_word:
              decoded_sentence += ' ' + sampled_word

          if (sampled_word == '<end>' or len(decoded_sentence.split()) > max_eng_len):
              stop_condition = True

          target_seq = np.zeros((1, 1))
          target_seq[0, 0] = sampled_token_index

      return decoded_sentence


In [ ]:
for seq_index in range(10):
    input_seq = rus_sequences[seq_index:seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('Input:', rus_sentences[seq_index])
    print('Decoded:', decoded_sentence)


1/1 [==============================] - 0s 49ms/step
Input: <start> том умнее, чем ты думаешь. <end>
Decoded:  2375818 end end end
1/1 [==============================] - 0s 54ms/step
Input: <start> что вы собираетесь делать? <end>
Decoded:  end end end end
1/1 [==============================] - 0s 55ms/step
Input: <start> я хочу поблагодарить тебя. <end>
Decoded:  3161734 end end end
1/1 [==============================] - 0s 35ms/step
Input: <start> это было ужасно. <end>
Decoded:  2255462 end end end
1/1 [==============================] - 0s 35ms/step
Input: <start> веди ты. <end>
Decoded:  2255462 end end end
1/1 [==============================] - 0s 35ms/step
Input: <start> чего я действительно не могу понять, так это зачем запрещать детям играть на улице в такой замечательный день. <end>
Decoded:  1936442 end end end
1/1 [==============================] - 0s 35ms/step
Input: <start> можем встретиться на следующей неделе, если хотите. <end>
Decoded:  276976 end end end
1/1 [=========